In [1]:
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

polarity_list = ["+", "-"]
polarity_distribution = [.5, .5] #distribution of positive and negative literals are equal

In [2]:
def random_sentence(atom, df_name_list, df_attribute_list):
  random_name = df_name_list.iloc[random.randint(0, len(df_name_list)-1)]['name']
  random_attribute = df_attribute_list.iloc[random.randint(0,len(df_attribute_list)-1)]['predicates']
  if (atom > 0):
    sentence = random_name + " is " + random_attribute
  else:
    sentence = random_name + " is not " + random_attribute
  return sentence

def flip_sentence(sent, atom):
  words = sent.split()
  if (atom > 0):
    sentence = words[0] + " is not " + words[2]
  else:
    sentence = words[0] + " is " + words[3]
  return sentence

def flip_atom(atm):
  atom = -atm
  return atom

def get_sentence(df, atm):
  sent = ""
  for i in range(len(df)):
    if(df['atom'][i]==atm):
      sent = df['sentence'][i]
  return str(sent)

In [3]:
import pandas as pd
import math

df_name_list = pd.read_csv('./dataset/baby-names.csv', sep=',', engine="python")
df_logic = pd.read_csv('./dataset/edited_rule_2sat2var_1.csv', sep=';', engine="python")
df_adjectives = pd.read_csv('./dataset/predicates.csv', sep=';', engine="python")

#Partition train : eval : test = 10 : 1 : 1
part_name = math.floor(1/12*len(df_name_list))
part_attribute = math.floor(1/12*len(df_adjectives))

train_name = df_name_list.iloc[:10*part_name, 1:2]
eval_name = df_name_list.iloc[10*part_name+1:11*part_name, 1:2]
test_name = df_name_list.iloc[11*part_name+1:, 1:2]

eval_name = eval_name.reset_index(drop=True)
test_name = test_name.reset_index(drop=True)

train_attribute = df_adjectives.iloc[:10*part_attribute, :1]
eval_attribute = df_adjectives.iloc[10*part_attribute+1:11*part_name, :1]
test_attribute = df_adjectives.iloc[11*part_attribute+1:, :1]

eval_attribute = eval_attribute.reset_index(drop=True)
test_attribute = test_attribute.reset_index(drop=True)

In [4]:
#test_name

In [5]:
#test_attribute

In [7]:
premises = []
idx = 0
data_train = 25
data_eval = 25000
data_test = 25000
df = pd.DataFrame(columns=['idx','is_inclusive','premise','question','label'])

name_list = train_name
attribute_list = train_attribute

for i in range(data_train):

  index_set = random.randint(0,len(df_logic)-1)
  sentence = [4,2]

  sentence_1 = random_sentence(df_logic['var1'][index_set], name_list, attribute_list)
  sentence_2 = random_sentence(df_logic['var2'][index_set], name_list, attribute_list)

  data = {'atom': [df_logic['var1'][index_set], flip_atom(df_logic['var1'][index_set]), df_logic['var2'][index_set], flip_atom(df_logic['var2'][index_set])], 
          'sentence': [sentence_1, flip_sentence(sentence_1, df_logic['var1'][index_set]),
                      sentence_2, flip_sentence(sentence_2, df_logic['var2'][index_set])]}  
  df_sentence = pd.DataFrame(data)

  df_sentence.head()

  #Generate rule sentence
  premise_sentence = ""
  if (df_logic['is_inclusive'][index_set]==1):
    premise_sentence = premise_sentence + sentence_1 + " or " + sentence_2 + ". "
  else:
    premise_sentence = premise_sentence + "Either " + sentence_1 + " or " + sentence_2 + ". "

  #Generate fact sentence
  #premises.append(df_sentence['sentence'].where(df_sentence['atom']==df_logic['f'][index_set]))
  premise_sentence = premise_sentence + get_sentence(df_sentence, df_logic['fact'][index_set]) + "."

  for i in range(4):
    idx = idx + 1
    c = 'c'+str(i+1)
    l = 'label'+str(i+1)
    q = get_sentence(df_sentence, df_logic[c][index_set])
    label = df_logic[l][index_set]
    full_sent = str(idx) + ";" + str(df_logic['is_inclusive'][index_set])+ ";" + premise_sentence + ";" + q + ";" + str(label)  
    df = df.append({'idx':str(idx),'is_inclusive':str(df_logic['is_inclusive'][index_set]),'premise':premise_sentence,'question':q,'label':str(label)}, ignore_index=True)

#df.head()

df.to_csv('train_data_1000K.csv', index=False)

C:\Users\Narul\AppData\Local\Temp\ipykernel_15872\3573615794.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'idx':str(idx),'is_inclusive':str(df_logic['is_inclusive'][index_set]),'premise':premise_sentence,'question':q,'label':str(label)}, ignore_index=True)
C:\Users\Narul\AppData\Local\Temp\ipykernel_15872\3573615794.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'idx':str(idx),'is_inclusive':str(df_logic['is_inclusive'][index_set]),'premise':premise_sentence,'question':q,'label':str(label)}, ignore_index=True)
C:\Users\Narul\AppData\Local\Temp\ipykernel_15872\3573615794.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'idx':str(idx),'is_inclusive':str(df_logi